In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [1]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_machine_learning.algorithms import VQC
from sklearn.preprocessing import LabelEncoder
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit_machine_learning.circuit.library import RawFeatureVector
from sklearn.preprocessing import MinMaxScaler


import numpy as np
import gym

# Carica il dataset Iris
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Caricamento dei dati
data = load_iris()
X = data['data']
y = data['target']

# Preprocessing
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2    , random_state=42, stratify=y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train)


# Creazione di un circuito iniziale

(120, 4) (120,) (30, 4) (30,)
[[0.02777778 0.375      0.06779661 0.04166667]
 [0.16666667 0.20833333 0.59322034 0.66666667]
 [0.69444444 0.33333333 0.6440678  0.54166667]
 [0.16666667 0.45833333 0.08474576 0.        ]
 [0.33333333 0.20833333 0.50847458 0.5       ]
 [0.55555556 0.20833333 0.6779661  0.75      ]
 [0.36111111 0.29166667 0.54237288 0.5       ]
 [0.55555556 0.33333333 0.69491525 0.58333333]
 [0.94444444 0.41666667 0.86440678 0.91666667]
 [0.94444444 0.75       0.96610169 0.875     ]
 [0.91666667 0.41666667 0.94915254 0.83333333]
 [0.47222222 0.375      0.59322034 0.58333333]
 [0.19444444 0.         0.42372881 0.375     ]
 [0.41666667 0.29166667 0.52542373 0.375     ]
 [0.41666667 0.25       0.50847458 0.45833333]
 [0.11111111 0.5        0.10169492 0.04166667]
 [0.22222222 0.75       0.08474576 0.08333333]
 [0.66666667 0.20833333 0.81355932 0.70833333]
 [0.58333333 0.45833333 0.76271186 0.70833333]
 [0.30555556 0.58333333 0.08474576 0.125     ]
 [0.55555556 0.20833333 0.6610

In [2]:
num_features = X.shape[1]

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2, entanglement="linear")
feature_map.draw(output='mpl')
feature_map.decompose().draw(output='mpl')
print("Feature map dimension:", feature_map.num_parameters)
print(feature_map)


Feature map dimension: 4
     ┌────────────────────────────────────┐
q_0: ┤0                                   ├
     │                                    │
q_1: ┤1                                   ├
     │  ZZFeatureMap(x[0],x[1],x[2],x[3]) │
q_2: ┤2                                   ├
     │                                    │
q_3: ┤3                                   ├
     └────────────────────────────────────┘


In [4]:
"""import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

class QNetwork(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_shape, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, num_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Creazione del modello
input_shape = 4  # Numero di features dell'input (es. dimensioni dell'osservazione)
num_actions = 2  # Numero di azioni possibili

model = QNetwork(input_shape, num_actions)

# Ottimizzatore e perdita
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
"""



"import torch\nimport torch.nn as nn\nimport torch.optim as optim\nimport torch.nn.functional as F\nimport numpy as np\n\nclass QNetwork(nn.Module):\n    def __init__(self, input_shape, num_actions):\n        super(QNetwork, self).__init__()\n        self.fc1 = nn.Linear(input_shape, 128)\n        self.fc2 = nn.Linear(128, 128)\n        self.fc3 = nn.Linear(128, num_actions)\n    \n    def forward(self, x):\n        x = F.relu(self.fc1(x))\n        x = F.relu(self.fc2(x))\n        return self.fc3(x)\n\n# Creazione del modello\ninput_shape = 4  # Numero di features dell'input (es. dimensioni dell'osservazione)\nnum_actions = 2  # Numero di azioni possibili\n\nmodel = QNetwork(input_shape, num_actions)\n\n# Ottimizzatore e perdita\noptimizer = optim.Adam(model.parameters(), lr=0.001)\nloss_fn = nn.MSELoss()\n"

In [5]:
"""class DQNAgent:
    def __init__(self, model, optimizer, loss_fn, num_actions, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, gamma=0.99):
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.num_actions = num_actions
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.gamma = gamma
        self.memory = []
        self.batch_size = 32
        self.max_memory_size = 1000
    
    def remember(self, state, action, reward, next_state, done):
        if len(self.memory) > self.max_memory_size:
            self.memory.pop(0)
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.num_actions)
        state = torch.FloatTensor(state).unsqueeze(0)
        q_values = self.model(state)
        return torch.argmax(q_values).item()
    
    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        
        batch = np.random.choice(len(self.memory), self.batch_size, replace=False)
        for i in batch:
            state, action, reward, next_state, done = self.memory[i]
            
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)
            target = reward
            
            if not done:
                target = reward + self.gamma * torch.max(self.model(next_state)).item()
            
            target_f = self.model(state)
            target_f[0][action] = target
            
            self.optimizer.zero_grad()
            output = self.model(state)
            loss = self.loss_fn(output, target_f)
            loss.backward()
            self.optimizer.step()
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
"""
            


'class DQNAgent:\n    def __init__(self, model, optimizer, loss_fn, num_actions, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, gamma=0.99):\n        self.model = model\n        self.optimizer = optimizer\n        self.loss_fn = loss_fn\n        self.num_actions = num_actions\n        self.epsilon = epsilon\n        self.epsilon_decay = epsilon_decay\n        self.epsilon_min = epsilon_min\n        self.gamma = gamma\n        self.memory = []\n        self.batch_size = 32\n        self.max_memory_size = 1000\n    \n    def remember(self, state, action, reward, next_state, done):\n        if len(self.memory) > self.max_memory_size:\n            self.memory.pop(0)\n        self.memory.append((state, action, reward, next_state, done))\n    \n    def act(self, state):\n        if np.random.rand() <= self.epsilon:\n            return np.random.choice(self.num_actions)\n        state = torch.FloatTensor(state).unsqueeze(0)\n        q_values = self.model(state)\n        return torch.argm

In [6]:
print(X_train[0])
print(y_train[0])

[0.02777778 0.375      0.06779661 0.04166667]
0


In [7]:
"""
import gym
import gym_quantcircuit
import numpy as np
import matplotlib as plt

env = gym.make('quantcircuit-v0')
agent = DQNAgent(model, optimizer, loss_fn, num_actions)

episodes = 100
num_qubits = 4
test_goal_state = [0j] * (2**num_qubits - 1) + [1+0j]
env.var_init(num_features,
             unitary=False,
             gate_group='pauli',
             connectivity='fully_connected',
             X_train=X_train,
             Y_train=y_train,
             X_test=X_test,
             Y_test=y_test,
             feature_map=feature_map,
             goal_state=test_goal_state)

for e in range(episodes):
    print(f"Episode {e+1}/{episodes}")
    state = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        
        action = env.sample()
        env.gate_list[action]
        print(env.gate_list[action])
        next_state, reward, done, _ = env.step(action)
        
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        
        if done:
            print(f"Episode {e+1}/{episodes} - Reward: {total_reward}")
            break
    
    agent.replay()
"""


'\nimport gym\nimport gym_quantcircuit\nimport numpy as np\nimport matplotlib as plt\n\nenv = gym.make(\'quantcircuit-v0\')\nagent = DQNAgent(model, optimizer, loss_fn, num_actions)\n\nepisodes = 100\nnum_qubits = 4\ntest_goal_state = [0j] * (2**num_qubits - 1) + [1+0j]\nenv.var_init(num_features,\n             unitary=False,\n             gate_group=\'pauli\',\n             connectivity=\'fully_connected\',\n             X_train=X_train,\n             Y_train=y_train,\n             X_test=X_test,\n             Y_test=y_test,\n             feature_map=feature_map,\n             goal_state=test_goal_state)\n\nfor e in range(episodes):\n    print(f"Episode {e+1}/{episodes}")\n    state = env.reset()\n    done = False\n    total_reward = 0\n    \n    while not done:\n        \n        action = env.sample()\n        env.gate_list[action]\n        print(env.gate_list[action])\n        next_state, reward, done, _ = env.step(action)\n        \n        \n        agent.remember(state, action, r

In [3]:
import gym
import gym_quantcircuit
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make('quantcircuit-v0')

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)


In [4]:

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)



In [5]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
    

In [6]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 5
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
test_goal_state = [0j] * (2**num_features - 1) + [1+0j]
env.var_init(num_qubits=num_features,
             unitary=False,
             gate_group='pauli',
             connectivity='fully_connected',
             X_train=X_train,
             Y_train=y_train,
             X_test=X_test,
             Y_test=y_test,
             feature_map=feature_map,
             goal_state=test_goal_state)

# Get number of actions from gym action space
n_actions = 2

# Get the number of state observations
state, info = env.reset()
n_observations = 1

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
        # Ensure state is 2D: (1, num_features)
    if len(state.shape) == 1:  # If it's a 1D tensor, add a batch dimension
                state = state.unsqueeze(0)
            
    if sample > eps_threshold:
                with torch.no_grad():
                    # Check state shape
                    print("State shape before passing to policy_net:", state.shape)
                    return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.sample()]], device=device, dtype=torch.long)




episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    print("Ottimizzazione del modello")
    
    # Extract a batch of transitions from the replay buffer
    transitions = memory.sample(BATCH_SIZE)
    
    # Convert the batch of transitions into a batch of arrays (from Transition of batch)
    batch = Transition(*zip(*transitions))

    # Concatenate the elements of the batch and check dimensions
    state_batch = torch.cat(batch.state).view(BATCH_SIZE, -1)

    # Ensure actions are scalars and convert them to 1D tensors
    action_batch = [a.view(-1) if a.dim() == 0 else a for a in batch.action]
    action_batch = torch.stack(action_batch).view(BATCH_SIZE, -1)
    
    reward_batch = torch.cat(batch.reward)
    
    # Calculate Q(s_t, a) using the policy network
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Calculate V(s_{t+1}) for all next states
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), 
                                  device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    # Ensure non_final_next_states has the correct shape
    if non_final_next_states.dim() == 1:
        non_final_next_states = non_final_next_states.unsqueeze(1)
    
    # Initialize next state values with zeros
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    
    # Calculate V(s_{t+1}) for non-terminal states
    with torch.no_grad():
        if len(non_final_next_states) > 0:
            next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values

    # Compute expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Calculate Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()

    # Clip gradients to prevent exploding gradients
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    
    # Optimizer step
    optimizer.step()

    print("Modello ottimizzato con successo!")

In [13]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 100
else:
    num_episodes = 50
episode_results=[]
for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    print("Episiodio numero: "+ str(i_episode+1)+"/"+str(num_episodes))
    state, info = env.reset()
    #print(info.get('current_accuracy'))
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        print(state.shape)
        action = select_action(state)
        observation, reward, terminated, _ = env.step(action)
        print("La reward sul test set è :"+str(reward))
        print("L' episiodio risulta essere: "+str(terminated))
        
        #reward = torch.tensor([reward], device=device)
        reward = torch.tensor([reward], dtype=torch.float32, device=device)

        done = terminated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        print("Lo stato :"+str(state))
        print("L'action è :"+str(action))
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        print("Pesi aggiornati")
        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break
        # Store the accuracy and rewards for this episode
        episode_results.append({
            "episode": i_episode,
            "final_reward": reward.item(),  # Convert to a regular float
            "duration": t + 1,
            "accuracy": info.get('current_accuracy')
        })

# Print all results after the loop
print(info.get('circuit'))
for result in episode_results:
    print(f"Episode {result['episode']}: Reward: {result['final_reward']}, Duration: {result['duration']}, Accuracy: {result['accuracy']}")


print('Complete')
#plot_durations(show_result=True)
#plt.ioff()
#plt.show()


Episiodio numero: 1/100
torch.Size([1])
State shape before passing to policy_net: torch.Size([1, 1])
      
q1_0: 
      
q1_1: 
      
q1_2: 
      
q1_3: 
      
Step count:  1
           
q1_0: ─────
      ┌───┐
q1_1: ┤ I ├
      └───┘
q1_2: ─────
           
q1_3: ─────
           
La reward sul test set è :0.0
L' episiodio risulta essere: False
Lo stato :tensor([0.], device='mps:0')
L'action è :tensor([[1]], device='mps:0')
Pesi aggiornati
torch.Size([1])
l'azione schelta è:24
           
q1_0: ─────
      ┌───┐
q1_1: ┤ I ├
      └───┘
q1_2: ─────
           
q1_3: ─────
           
Step count:  2
           
q1_0: ─────
      ┌───┐
q1_1: ┤ I ├
      └───┘
q1_2: ──■──
      ┌─┴─┐
q1_3: ┤ X ├
      └───┘
La reward sul test set è :0.0
L' episiodio risulta essere: False
Lo stato :tensor([0.], device='mps:0')
L'action è :tensor([[24]], device='mps:0')
Pesi aggiornati
torch.Size([1])
l'azione schelta è:24
           
q1_0: ─────
      ┌───┐
q1_1: ┤ I ├
      └───┘
q1_2: ──■──
      ┌─┴

In [14]:
# Salva il modello addestrato
torch.save({
    'policy_net_state_dict': policy_net.state_dict(),
    'target_net_state_dict': target_net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'rl_agent.pth')
print("Modello salvato con successo!")


Modello salvato con successo!


In [1]:
# Ricarica il modello salvato
checkpoint = torch.load('rl_agent.pth')
policy_net.load_state_dict(checkpoint['policy_net_state_dict'])
target_net.load_state_dict(checkpoint['target_net_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print("Modello ricaricato con successo!")
print(policy_net)


NameError: name 'torch' is not defined

In [10]:
def generate_quantum_circuit(agent, env):
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    done = False
    while not done:
        action = select_action(state)
        state, reward, done, _ = env.step(action)
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

# Genera il circuito quantistico
test_goal_state = [0j] * (2**num_features - 1) + [1+0j]
env.var_init(num_qubits=num_features,
             unitary=False,
             gate_group='pauli',
             connectivity='fully_connected',
             X_train=X_train,
             Y_train=y_train,
             X_test=X_test,
             Y_test=y_test,
             feature_map=feature_map,
             goal_state=test_goal_state)

quantum_circuit = generate_quantum_circuit(policy_net, env)
print("Circuito quantistico generato con successo!")


l'azione schelta è:8
      
q2_0: 
      
q2_1: 
      
q2_2: 
      
q2_3: 
      
Step count:  1
      ┌───┐
q2_0: ┤ X ├
      └───┘
q2_1: ─────
           
q2_2: ─────
           
q2_3: ─────
           
l'azione schelta è:6
      ┌───┐
q2_0: ┤ X ├
      └───┘
q2_1: ─────
           
q2_2: ─────
           
q2_3: ─────
           
Step count:  2
      ┌───┐
q2_0: ┤ X ├
      └───┘
q2_1: ─────
      ┌───┐
q2_2: ┤ H ├
      └───┘
q2_3: ─────
           
l'azione schelta è:24
      ┌───┐
q2_0: ┤ X ├
      └───┘
q2_1: ─────
      ┌───┐
q2_2: ┤ H ├
      └───┘
q2_3: ─────
           
Step count:  3
      ┌───┐     
q2_0: ┤ X ├─────
      └───┘     
q2_1: ──────────
      ┌───┐     
q2_2: ┤ H ├──■──
      └───┘┌─┴─┐
q2_3: ─────┤ X ├
           └───┘
l'azione schelta è:25
      ┌───┐     
q2_0: ┤ X ├─────
      └───┘     
q2_1: ──────────
      ┌───┐     
q2_2: ┤ H ├──■──
      └───┘┌─┴─┐
q2_3: ─────┤ X ├
           └───┘
Step count:  4
      ┌───┐     ┌───┐
q2_0: ┤ X ├─────┤ X ├
      └──